In [1]:
from keras.models import load_model
enc_model = load_model('encoder_model', compile=False)
dec_model = load_model('decoder_model', compile=False)

In [2]:
import pickle

inv_vocab = pickle.load(open("vocabulary.pkl", "rb"))
vocab = {w:v for v, w in inv_vocab.items()}
keyword_list = pickle.load(open("keyword.pkl", "rb"))
enc_in = pickle.load(open("question.pkl", "rb"))

In [3]:
import numpy as np
from laonlp import word_tokenize
import re
from keras.preprocessing.sequence import pad_sequences

In [4]:
def remove_special_character(text):
    return re.sub(r'[^0-9a-zA-Zກຂຄງຈສຊຍດຕຖທນບປຜຝພຟມຢລຫຼຣວຫອຮໜໝໆຽະາ ິ ີ ຶ ື ໂ ໍເແ ຸ ູຳໄໃ ັ ົ ່ ້ ໌+]', '', text)

def remove_spaces(text):
    
    return text.replace(" ", "")

In [5]:
def clean_text(txt):
    txt = re.sub(r"a", "A", txt)
    txt = re.sub(r"b", "B", txt)
    txt = re.sub(r"b+", "B+", txt)
    txt = re.sub(r"c+", "C+", txt)
    txt = re.sub(r"c", "C", txt)
    txt = re.sub(r"d+", "D+", txt)
    txt = re.sub(r"d", "D", txt)
    txt = re.sub(r"f", "F", txt)
    txt = re.sub(r"ai", "AI", txt)
    txt = re.sub(r"ນານາ", "ນາໆ", txt)
    txt = re.sub(r"ພາຍຫລັງ", "ພາຍຫຼັງ", txt)
    txt = re.sub(r"ຫົວຫນ້າ", "ຫົວໜ້າ", txt)
    txt = re.sub(r"ຫລັກສູດ", "ຫຼັກສູດ", txt)
    return txt

In [6]:
def decode_sequence(input_seq):
    for sentence in enc_in:
        if np.array_equal(input_seq, [sentence]):
            states_value = enc_model.predict(input_seq)

            target_seq = np.zeros((1,1))
            target_seq[0,0] = vocab['<SOS>']

            stop_condition = False
            decoded_sentence = ''
            while not stop_condition:
                output_tokens, h, c = dec_model.predict([target_seq] + states_value)
                sampled_token_index = np.argmax(output_tokens[0, -1, :])
                sampled_word = inv_vocab[sampled_token_index]
                decoded_sentence += '' + sampled_word

                if(sampled_word == '<EOS>' or len(word_tokenize(decoded_sentence)) > 100):
                    stop_condition = True

                target_seq = np.zeros((1,1))
                target_seq[0,0] = sampled_token_index
                
                states_value = [h,c]

            return decoded_sentence
    return "ຂໍອະໄພ, ບໍ່ສາມາດຕອບຄຳຖາມນີ້ໄດ້"  

In [8]:
#TEST
input_text = "ເກຣດaຕ້ອງມີຄະແນນເທົ່າໃດ"
input_seq = clean_text(input_text)
input_seq = remove_special_character(input_seq)
input_seq = remove_spaces(input_seq)
txt = []
lst = []
input_seq = word_tokenize(input_seq)
words = []
for word in input_seq:
    if word in keyword_list:
        words.append(word)
for x in words:
    try:
        lst.append(vocab[x])
    except:
        lst.append(vocab['<OUT>'])
    
txt.append(lst)

input_padded_seq = pad_sequences(txt, 15, padding='post', truncating='post')
response = decode_sequence(input_padded_seq)
response = response.replace('<EOS>', '')
print(f"You: {input_text}")
print(f"Bot: {response}")

1/1 [==============================] - 0s 25ms/step
You: ເກຣດaຕ້ອງມີຄະແນນເທົ່າໃດ
Bot: ເກຣດ A ຕ້ອງມີຄະແນນຕັ້ງແຕ່ 91 ຄະແນນຂຶ້ນໄປ


In [9]:
while True:
    input_text = input("You: ")
    if input_text == 'exit':
        break
    input_seq = remove_special_character(input_text)
    input_seq = remove_spaces(input_seq)
    txt = []
    lst = []
    input_seq = word_tokenize(input_seq)
    words = [word for word in input_seq if word in keyword_list]
            
    for x in words:
        try:
            lst.append(vocab[x])
        except:
            lst.append(vocab['<OUT>'])
    
    txt.append(lst)
    
    input_padded_seq = pad_sequences(txt, 15, padding='post', truncating='post')
    response = decode_sequence(input_padded_seq)
    print(f"You: {input_text} / {input_padded_seq}")
    print(f"Bot: {response}")

1/1 [==============================] - 0s 20ms/step
You: ສາຂາການພັດທະນາເວັບໄຊໃຊ້ເວລາຮຽນຈັກມື້ຕໍ່ອາທິດ / [[523 440 627 586 283 463   0   0   0   0   0   0   0   0   0]]
Bot: ຫຼັກສູດປະລິນຍາຕີ ສາຂາການພັດທະນາເວັບໄຊ ແມ່ນໄດ້ຮຽນ 5 ມື້ຕໍ່ອາທິດ (ວັນຈັນ ຫາ ວັນສຸກ)<EOS>
1/1 [==============================] - 0s 20ms/step
You: ສາຂາການພັດທະນາເວັບໄຊມີການຈັດການໄປທັດສະນະສຶກສາບໍ / [[523 440 627 356 531   0   0   0   0   0   0   0   0   0   0]]
Bot: ການໄປທັດສະນະສຶກສາໃນທຸກສາຂາຂອງຫຼັກສູດປະລິນຍາຕີແມ່ນຈະຕ້ອງຜ່ານການຢັ້ງຢືນຈາກຄະນະວິຊາ ແລະ ຄ່າໃຊ້ຈ່າຍໃນການໄປທັດສະນະສຶກສານັກສຶກສາຈະຕ້ອງຮັບຜິດຊອບເອງ<EOS>


In [8]:
len(keyword_list)

96